In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Machine Learning Fundamentals with BigQuery DataFrames

<table align="left">

   <td>
    <a href="https://github.com/googleapis/python-bigquery-dataframes/blob/main/notebooks/getting_started/ml_fundamentals_bq_dataframes.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
                                                                                       
</table>

**_NOTE_**: This notebook has been tested in the following environment:

* Python version = 3.10

## Overview

The `bigframes.ml` module implements Scikit-Learn's machine learning API in
BigQuery DataFrames. It exposes BigQuery's ML capabilities in a simple, popular
API that works seamlessly with the rest of the BigQuery DataFrames API.

Learn more about [BigQuery DataFrames](https://cloud.google.com/python/docs/reference/bigframes/latest).

### Objective

In this tutorial, you will walk through an end-to-end machine learning workflow using BigQuery DataFrames. You will load data, manipulate and prepare it for model training, build supervised and unsupervised models, and evaluate and save a model for future use; all using built-in BigQuery DataFrames functionality.

### Dataset

This tutorial uses the [```penguins``` table](https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=ml_datasets&t=penguins) (a BigQuery public dataset), which contains data on a set of penguins including species, island of residence, weight, culmen length and depth, flipper length, and sex.

### Costs

This tutorial uses billable components of Google Cloud:

* BigQuery (storage and compute)
* BigQuery ML

Learn about [BigQuery storage pricing](https://cloud.google.com/bigquery/pricing#storage),
[BigQuery compute pricing](https://cloud.google.com/bigquery/pricing#analysis_pricing_models),
and [BigQuery ML pricing](https://cloud.google.com/bigquery/pricing#bqml),
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Installation

Depending on your Jupyter environment, you might have to install packages.

**Vertex AI Workbench or Colab**

Do nothing, BigQuery DataFrames package is already installed.

**Local JupyterLab instance**

Uncomment and run the following cell:

In [ ]:
# !pip install bigframes

## Before you begin

Complete the tasks in this section to set up your environment.

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

3. [Click here](https://console.cloud.google.com/flows/enableapi?apiid=bigquery.googleapis.com) to enable the BigQuery API.

4. If you are running this notebook locally, install the [Cloud SDK](https://cloud.google.com/sdk).

#### Set your project ID

If you don't know your project ID, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113).

In [1]:
PROJECT_ID = "jrdetorre-bq-demo"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

Updated property [core/project].


#### Set the region

You can also change the `REGION` variable used by BigQuery. Learn more about [BigQuery regions](https://cloud.google.com/bigquery/docs/locations#supported_locations).

In [2]:
REGION = "US"  # @param {type: "string"}

#### Set the dataset ID

As part of this notebook, you will save BigQuery ML models to your Google Cloud project, which requires a dataset. Create the dataset, if needed, and provide the ID here as the `DATASET` variable used by BigQuery. Learn how to create a [BigQuery dataset](https://cloud.google.com/bigquery/docs/datasets).

In [3]:
DATASET = "birds"  # @param {type: "string"}

### Import libraries

In [4]:
import bigframes.pandas as bpd


### Set BigQuery DataFrames options

In [6]:
# Note: The project option is not required in all environments.
# On BigQuery Studio, the project ID is automatically detected.
bpd.options.bigquery.project = PROJECT_ID

# Note: The location option is not required.
# It defaults to the location of the first table or query
# passed to read_gbq(). For APIs where a location can't be
# auto-detected, the location defaults to the "US" location.
bpd.options.bigquery.location = REGION

If you want to reset the location of the created DataFrame or Series objects, reset the session by executing `bpd.reset_session()`. After that, you can reuse `bpd.options.bigquery.location` to specify another location.

## Import data into BigQuery DataFrames

You can create a DataFrame by reading data from a BigQuery table.

In [7]:
df = bpd.read_gbq("bigquery-public-data.ml_datasets.penguins")
df = df.dropna()

# BigQuery DataFrames creates a default numbered index, which we can give a name
df.index.name = "penguin_id"

Take a look at a few rows of the DataFrame:

In [8]:
df.head()

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
penguin_id,,,,,,,
0,Gentoo penguin (Pygoscelis papua),Biscoe,50.5,15.9,225.0,5400.0,MALE
1,Gentoo penguin (Pygoscelis papua),Biscoe,45.1,14.5,215.0,5000.0,FEMALE
2,Adelie Penguin (Pygoscelis adeliae),Torgersen,41.4,18.5,202.0,3875.0,MALE
3,Adelie Penguin (Pygoscelis adeliae),Torgersen,38.6,17.0,188.0,2900.0,FEMALE
4,Gentoo penguin (Pygoscelis papua),Biscoe,46.5,14.8,217.0,5200.0,FEMALE


## Clean and prepare data

We're are going to start with supervised learning, where a Linear Regression model will learn to predict the body mass (output variable `y`) using input features such as flipper length, sex, species, and more (features `X`).

In [9]:
# Isolate input features and output variable into DataFrames
X = df[['island', 'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'sex', 'species']]
y = df[['body_mass_g']]

Part of preparing data for a machine learning task is splitting it into subsets for training and testing to ensure that the solution is not overfitting. By default, BQML will automatically manage splitting the data for you. However, BQML also supports manually splitting out your training data.

Performing a manual data split can be done with `bigframes.ml.model_selection.train_test_split` like so:

In [10]:
from bigframes.ml.model_selection import train_test_split

# This will split X and y into test and training sets, with 20% of the rows in the test set,
# and the rest in the training set
X_train, X_test, y_train, y_test = train_test_split(
  X, y, test_size=0.2)

# Show the shape of the data after the split
print(f"""X_train shape: {X_train.shape}
X_test shape: {X_test.shape}
y_train shape: {y_train.shape}
y_test shape: {y_test.shape}""")

X_train shape: (267, 6)
X_test shape: (67, 6)
y_train shape: (267, 1)
y_test shape: (67, 1)


If we look at the data, we can see that random rows were selected for
each side of the split:

In [11]:
X_test.head(5)

,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,sex,species
penguin_id,,,,,,
47,Dream,37.5,18.5,199.0,MALE,Adelie Penguin (Pygoscelis adeliae)
144,Biscoe,46.5,14.5,213.0,FEMALE,Gentoo penguin (Pygoscelis papua)
75,Biscoe,37.8,20.0,190.0,MALE,Adelie Penguin (Pygoscelis adeliae)
67,Biscoe,47.6,14.5,215.0,MALE,Gentoo penguin (Pygoscelis papua)
172,Biscoe,48.2,15.6,221.0,MALE,Gentoo penguin (Pygoscelis papua)


Note that the `y_test` data matches the same rows in `X_test`:

In [12]:
y_test.head(5)

,body_mass_g
penguin_id,
47,4475.0
144,4400.0
75,4250.0
67,5400.0
172,5100.0


## Estimators

Following scikit-learn, all learning components are "estimators"; objects that can learn from training data and then apply themselves to new data. Estimators share the following patterns:

- a constructor that takes a list of parameters
- a standard string representation that shows the class name and all non-default parameters, e.g. `LinearRegression(fit_intercept=False)`
- a `.fit(..)` method to fit the estimator to training data

There estimators can be further broken down into two main subtypes:
 1. Transformers
 2. Predictors

Let's walk through each of these with our example model.

### Transformers

Transformers are estimators that are used to prepare data for consumption by other estimators ('preprocessing'). In addition to `.fit(...)`, the transformer implements a `.transform(...)` method, which will apply a transformation based on what was computed during `.fit(..)`. With this pattern dynamic preprocessing steps can be applied to both training and test/production data consistently.

An example of a transformer is `bigframes.ml.preprocessing.StandardScaler`, which rescales a dataset to have a mean of zero and a standard deviation of one:

In [16]:
from bigframes.ml.preprocessing import StandardScaler

# StandardScaler will only work on numeric columns
numeric_columns = ["culmen_length_mm", "culmen_depth_mm", "flipper_length_mm"]

scaler = StandardScaler()
scaler.fit(X_train[numeric_columns])

# Now, standardscaler should transform the numbers to have mean of zero
# and standard deviation of one:
scaler.transform(X_train[numeric_columns])

StandardScaler()

We can then repeat this transformation on the test data:

In [15]:
scaler.transform(X_test[numeric_columns])

,standard_scaled_culmen_length_mm,standard_scaled_culmen_depth_mm,standard_scaled_flipper_length_mm
penguin_id,,,
2,-0.499551,0.668843,0.054287
8,0.79789,-1.547704,1.340038
14,0.505509,-0.288302,-0.660019
16,0.925806,-0.993567,1.411469
20,-0.243718,-1.497328,0.554302
22,-1.248778,0.316211,-0.731449
41,-0.718837,-0.086798,-1.802909
45,1.437473,1.777116,0.625732
47,-1.21223,0.668843,-0.160005


#### Composing transformers

To process data where different columns need different preprocessors, `bigframes.composition.ColumnTransformer` can be employed.

Let's create an aggregate transform that applies `StandardScalar` to the numeric columns and `OneHotEncoder` to the string columns.

In [17]:
from bigframes.ml.compose import ColumnTransformer
from bigframes.ml.preprocessing import OneHotEncoder

# Create an aggregate transform that applies StandardScaler to the numeric columns,
# and OneHotEncoder to the string columns
preproc = ColumnTransformer([
    ("scale", StandardScaler(), ["culmen_length_mm", "culmen_depth_mm", "flipper_length_mm"]),
    ("encode", OneHotEncoder(), ["species", "sex", "island"])])

# Now we can fit all columns of the training data
preproc.fit(X_train)

processed_X_train = preproc.transform(X_train)
processed_X_test = preproc.transform(X_test)

# View the processed training data
processed_X_train

,standard_scaled_culmen_length_mm,standard_scaled_culmen_depth_mm,standard_scaled_flipper_length_mm,onehotencoded_species,onehotencoded_sex,onehotencoded_island
penguin_id,,,,,,
0,1.163366,-0.640935,1.697191,"[{'index': 3, 'value': 1.0}]","[{'index': 3, 'value': 1.0}]","[{'index': 1, 'value': 1.0}]"
1,0.17658,-1.346199,0.982885,"[{'index': 3, 'value': 1.0}]","[{'index': 2, 'value': 1.0}]","[{'index': 1, 'value': 1.0}]"
3,-1.011218,-0.086798,-0.945741,"[{'index': 1, 'value': 1.0}]","[{'index': 2, 'value': 1.0}]","[{'index': 3, 'value': 1.0}]"
4,0.432413,-1.195071,1.125746,"[{'index': 3, 'value': 1.0}]","[{'index': 2, 'value': 1.0}]","[{'index': 1, 'value': 1.0}]"
5,-1.669076,0.366587,-0.660019,"[{'index': 1, 'value': 1.0}]","[{'index': 2, 'value': 1.0}]","[{'index': 1, 'value': 1.0}]"
7,-0.389908,-1.84996,0.625732,"[{'index': 3, 'value': 1.0}]","[{'index': 2, 'value': 1.0}]","[{'index': 1, 'value': 1.0}]"
9,0.304497,0.870347,-0.302866,"[{'index': 1, 'value': 1.0}]","[{'index': 3, 'value': 1.0}]","[{'index': 3, 'value': 1.0}]"
10,0.889259,1.22298,0.768593,"[{'index': 2, 'value': 1.0}]","[{'index': 3, 'value': 1.0}]","[{'index': 2, 'value': 1.0}]"
11,-1.504611,-0.288302,-0.80288,"[{'index': 1, 'value': 1.0}]","[{'index': 2, 'value': 1.0}]","[{'index': 3, 'value': 1.0}]"


### Predictors

Predictors are estimators that learn and make predictions. In addition to `.fit(...)`, the predictor implements a `.predict(...)` method, which will use what was learned during `.fit(...)` to predict some output.

Predictors can be further broken down into two categories:
* Supervised predictors
* Unsupervised predictors

#### Supervised predictors

Supervised learning is when we train a model on input-output pairs, and then ask it to predict the output for new inputs. An example of such a predictor is `bigframes.ml.linear_models.LinearRegression`.

In [18]:
from bigframes.ml.linear_model import LinearRegression

linreg = LinearRegression()

# Learn from the training data how to predict output y
linreg.fit(processed_X_train, y_train)

# Predict y for the test data
predicted_y_test = linreg.predict(processed_X_test)

# View predictions
predicted_y_test

,predicted_body_mass_g,standard_scaled_culmen_length_mm,standard_scaled_culmen_depth_mm,standard_scaled_flipper_length_mm,onehotencoded_species,onehotencoded_sex,onehotencoded_island
penguin_id,,,,,,,
2,4173.408558,-0.499551,0.668843,0.054287,"[{'index': 1, 'value': 1.0}]","[{'index': 3, 'value': 1.0}]","[{'index': 3, 'value': 1.0}]"
8,5286.011999,0.79789,-1.547704,1.340038,"[{'index': 3, 'value': 1.0}]","[{'index': 3, 'value': 1.0}]","[{'index': 1, 'value': 1.0}]"
14,3366.800859,0.505509,-0.288302,-0.660019,"[{'index': 2, 'value': 1.0}]","[{'index': 2, 'value': 1.0}]","[{'index': 2, 'value': 1.0}]"
16,5385.634381,0.925806,-0.993567,1.411469,"[{'index': 3, 'value': 1.0}]","[{'index': 3, 'value': 1.0}]","[{'index': 1, 'value': 1.0}]"
20,4644.566555,-0.243718,-1.497328,0.554302,"[{'index': 3, 'value': 1.0}]","[{'index': 2, 'value': 1.0}]","[{'index': 1, 'value': 1.0}]"
22,3531.908843,-1.248778,0.316211,-0.731449,"[{'index': 1, 'value': 1.0}]","[{'index': 2, 'value': 1.0}]","[{'index': 2, 'value': 1.0}]"
41,3260.88653,-0.718837,-0.086798,-1.802909,"[{'index': 1, 'value': 1.0}]","[{'index': 2, 'value': 1.0}]","[{'index': 3, 'value': 1.0}]"
45,4376.494231,1.437473,1.777116,0.625732,"[{'index': 2, 'value': 1.0}]","[{'index': 3, 'value': 1.0}]","[{'index': 2, 'value': 1.0}]"
47,4063.330129,-1.21223,0.668843,-0.160005,"[{'index': 1, 'value': 1.0}]","[{'index': 3, 'value': 1.0}]","[{'index': 2, 'value': 1.0}]"


Quick check of the quality of the estimator

In [24]:
y_y=predicted_y_test
y_y['real_value']=y_test['body_mass_g']
y_y['error']=100*((y_test['body_mass_g']-predicted_y_test['predicted_body_mass_g']).abs())/y_test['body_mass_g']
y_y[["predicted_body_mass_g","real_value","error"]].head(5)

,predicted_body_mass_g,real_value,error
penguin_id,,,
2,4173.408558,3875.0,7.700866
8,5286.011999,5300.0,0.263925
14,3366.800859,2700.0,24.696328
16,5385.634381,6300.0,14.51374
20,4644.566555,4700.0,1.179435


#### Unsupervised predictors

In unsupervised learning, there are no known outputs in the training data, instead the model learns on input data alone and predicts something else. An example of an unsupervised predictor is `bigframes.ml.cluster.KMeans`, which learns how to fit input data to a target number of clusters.

In [25]:
from bigframes.ml.cluster import KMeans

# Specify KMeans with four clusters
kmeans = KMeans(n_clusters=4)

# Fit data
kmeans.fit(processed_X_train)

# View predictions
kmeans.predict(processed_X_test)

,CENTROID_ID,NEAREST_CENTROIDS_DISTANCE,standard_scaled_culmen_length_mm,standard_scaled_culmen_depth_mm,standard_scaled_flipper_length_mm,onehotencoded_species,onehotencoded_sex,onehotencoded_island
penguin_id,,,,,,,,
2,4,"[{'CENTROID_ID': 4, 'DISTANCE': 1.610888069891...",-0.499551,0.668843,0.054287,"[{'index': 1, 'value': 1.0}]","[{'index': 3, 'value': 1.0}]","[{'index': 3, 'value': 1.0}]"
8,2,"[{'CENTROID_ID': 2, 'DISTANCE': 1.880800394823...",0.79789,-1.547704,1.340038,"[{'index': 3, 'value': 1.0}]","[{'index': 3, 'value': 1.0}]","[{'index': 1, 'value': 1.0}]"
14,1,"[{'CENTROID_ID': 1, 'DISTANCE': 2.039159413717...",0.505509,-0.288302,-0.660019,"[{'index': 2, 'value': 1.0}]","[{'index': 2, 'value': 1.0}]","[{'index': 2, 'value': 1.0}]"
16,2,"[{'CENTROID_ID': 2, 'DISTANCE': 1.779245110306...",0.925806,-0.993567,1.411469,"[{'index': 3, 'value': 1.0}]","[{'index': 3, 'value': 1.0}]","[{'index': 1, 'value': 1.0}]"
20,2,"[{'CENTROID_ID': 2, 'DISTANCE': 2.030063115528...",-0.243718,-1.497328,0.554302,"[{'index': 3, 'value': 1.0}]","[{'index': 2, 'value': 1.0}]","[{'index': 1, 'value': 1.0}]"
22,4,"[{'CENTROID_ID': 4, 'DISTANCE': 1.583547035187...",-1.248778,0.316211,-0.731449,"[{'index': 1, 'value': 1.0}]","[{'index': 2, 'value': 1.0}]","[{'index': 2, 'value': 1.0}]"
41,4,"[{'CENTROID_ID': 4, 'DISTANCE': 1.916057303822...",-0.718837,-0.086798,-1.802909,"[{'index': 1, 'value': 1.0}]","[{'index': 2, 'value': 1.0}]","[{'index': 3, 'value': 1.0}]"
45,1,"[{'CENTROID_ID': 1, 'DISTANCE': 1.998768779704...",1.437473,1.777116,0.625732,"[{'index': 2, 'value': 1.0}]","[{'index': 3, 'value': 1.0}]","[{'index': 2, 'value': 1.0}]"
47,4,"[{'CENTROID_ID': 4, 'DISTANCE': 1.294346426101...",-1.21223,0.668843,-0.160005,"[{'index': 1, 'value': 1.0}]","[{'index': 3, 'value': 1.0}]","[{'index': 2, 'value': 1.0}]"


## Pipelines

Transfomers and predictors can be chained into a single estimator component using `bigframes.ml.pipeline.Pipeline`:

In [26]:
from bigframes.ml.pipeline import Pipeline

pipeline = Pipeline([
  ('preproc', preproc),
  ('linreg', linreg)
])

# Print our pipeline
pipeline

Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('scale', StandardScaler(),
                                                  ['culmen_length_mm',
                                                   'culmen_depth_mm',
                                                   'flipper_length_mm']),
                                                 ('encode', OneHotEncoder(),
                                                  ['species', 'sex',
                                                   'island'])])),
                ('linreg', LinearRegression())])

The pipeline simplifies the workflow by applying each of its component steps automatically:

In [27]:
pipeline.fit(X_train, y_train)

predicted_y_test = pipeline.predict(X_test)
predicted_y_test

,predicted_body_mass_g,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,sex,species
penguin_id,,,,,,,
2,4142.212801,Torgersen,41.4,18.5,202.0,MALE,Adelie Penguin (Pygoscelis adeliae)
8,5254.339302,Biscoe,48.5,14.1,220.0,MALE,Gentoo penguin (Pygoscelis papua)
14,3336.766261,Dream,46.9,16.6,192.0,FEMALE,Chinstrap penguin (Pygoscelis antarctica)
16,5356.522154,Biscoe,49.2,15.2,221.0,MALE,Gentoo penguin (Pygoscelis papua)
20,4615.713161,Biscoe,42.8,14.2,209.0,FEMALE,Gentoo penguin (Pygoscelis papua)
22,3501.46194,Dream,37.3,17.8,191.0,FEMALE,Adelie Penguin (Pygoscelis adeliae)
41,3235.60438,Torgersen,40.2,17.0,176.0,FEMALE,Adelie Penguin (Pygoscelis adeliae)
45,4355.645535,Dream,52.0,20.7,210.0,MALE,Chinstrap penguin (Pygoscelis antarctica)
47,4037.084173,Dream,37.5,18.5,199.0,MALE,Adelie Penguin (Pygoscelis adeliae)


In the backend, a pipeline will actually be compiled into a single model with an embedded TRANSFORM step.

## Evaluating results

Some models include a convenient `.score(X, y)` method for evaulation with a preset accuracy metric:

In [28]:
# In the case of a pipeline, this will be equivalent to calling .score on the contained LinearRegression
pipeline.score(X_test, y_test)

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,245.888023,95401.773593,0.006126,209.063178,0.878158,0.878916


For a more general approach, the library `bigframes.ml.metrics` is provided:

In [29]:
from bigframes.ml.metrics import r2_score

r2_score(y_test, predicted_y_test["predicted_body_mass_g"])

0.878158455094941

## Save to BigQuery

Estimators can be saved to BigQuery as BQML models, and loaded again in future.

Saving requires `bigquery.tables.create` permission, and loading requires `bigquery.models.getMetadata` permission.
These permissions can be at project level or the dataset level.

If you have those permissions, please go ahead and uncomment the code in the following cells and run.

In [30]:
linreg.to_gbq(f"{DATASET}.penguins_model", replace=True)

Pipeline(steps=[('transform',
                 ColumnTransformer(transformers=[('standard_scaler',
                                                  StandardScaler(),
                                                  'flipper_length_mm'),
                                                 ('standard_scaler',
                                                  StandardScaler(),
                                                  'culmen_depth_mm'),
                                                 ('one_hot_encoder',
                                                  OneHotEncoder(max_categories=1000001,
                                                                min_frequency=0),
                                                  'species'),
                                                 ('one_hot_encoder',
                                                  OneHotEncoder(max_categories=1000001,
                                                                min_frequency=0),
              

In [31]:
bpd.read_gbq_model(f"{DATASET}.penguins_model")

Pipeline(steps=[('transform',
                 ColumnTransformer(transformers=[('standard_scaler',
                                                  StandardScaler(),
                                                  'flipper_length_mm'),
                                                 ('standard_scaler',
                                                  StandardScaler(),
                                                  'culmen_depth_mm'),
                                                 ('one_hot_encoder',
                                                  OneHotEncoder(max_categories=1000001,
                                                                min_frequency=0),
                                                  'species'),
                                                 ('one_hot_encoder',
                                                  OneHotEncoder(max_categories=1000001,
                                                                min_frequency=0),
              

We can also save the pipeline to BigQuery. BigQuery will save this as a single model, with the pre-processing steps embedded in the TRANSFORM property:

In [32]:
pipeline.to_gbq(f"{DATASET}.penguins_pipeline", replace=True)

Pipeline(steps=[('transform',
                 ColumnTransformer(transformers=[('standard_scaler',
                                                  StandardScaler(),
                                                  'flipper_length_mm'),
                                                 ('standard_scaler',
                                                  StandardScaler(),
                                                  'culmen_depth_mm'),
                                                 ('one_hot_encoder',
                                                  OneHotEncoder(max_categories=1000001,
                                                                min_frequency=0),
                                                  'species'),
                                                 ('one_hot_encoder',
                                                  OneHotEncoder(max_categories=1000001,
                                                                min_frequency=0),
              

In [58]:
bpd.read_gbq_model(f"{DATASET}.penguins_pipeline")

Pipeline(steps=[('transform',
                 ColumnTransformer(transformers=[('standard_scaler',
                                                  StandardScaler(),
                                                  'culmen_depth_mm'),
                                                 ('standard_scaler',
                                                  StandardScaler(),
                                                  'flipper_length_mm'),
                                                 ('one_hot_encoder',
                                                  OneHotEncoder(max_categories=1000001,
                                                                min_frequency=0),
                                                  'sex'),
                                                 ('one_hot_encoder',
                                                  OneHotEncoder(max_categories=1000001,
                                                                min_frequency=0),
                  

## Summary and next steps

You've completed an end-to-end machine learning workflow using the built-in capabilities of BigQuery DataFrames.

Learn more about BigQuery DataFrames in the [documentation](https://cloud.google.com/python/docs/reference/bigframes/latest) and find more sample notebooks in the [GitHub repo](https://github.com/googleapis/python-bigquery-dataframes/tree/main/notebooks).

### Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can uncomment the remaining cells and run them to delete the individual resources you created in this tutorial:

In [ ]:
# # Delete the BQML models
# MODEL_NAME = f"{PROJECT_ID}:{DATASET}.penguins_model"
# ! bq rm -f --model {MODEL_NAME}
# PIPELINE_NAME = f"{PROJECT_ID}:{DATASET}.penguins_pipeline"
# ! bq rm -f --model {PIPELINE_NAME}